In [1]:
import numpy as numpy
import cv2
import os.path
import os
import statistics 
import mysql.connector
import csv


In [10]:
#img = cv2.imread('./imgs/img_416.png',0);
IMG_PATH = 'C:/models/ryuboxes/training'; #'C:/Users/jesus/source/repos/python1/python1/imgs'; # 
IMG_PATH_CONVERTED = 'C:/models/ryuboxes/training/test/tiles'; #'C:/Users/jesus/source/repos/python1/python1/imgs/converted'; # 'C:/models/ryumovs/training';
TILE_WIDTH = 16;
TILE_HEIGHT = 10;
GENERATE_COLOR_TILES = 0;
CSV_PATH = 'tilesetAvgUpdated.csv';

In [29]:
#cv2.imshow('image',img);
onlyfiles = [f for f in os.listdir(IMG_PATH) if os.path.isfile(IMG_PATH+'/'+f)]
   

def isBlackLine(row,cols):
    
    color = 0;
    
    for y in range(cols):
        if (row[y]!=0):
            color = color + 1;
    
    if (color<10):
        return True;
    
    return False;

def getBlackVerticalLines(img):           

    color = 0;
    lines = [];
    cols = img.shape[1];
    rows = img.shape[0];
        
    for c in range(cols):            
        for r in range(rows):        
            if (img[r,c]!=0):
                color=color+1;
            
            if (color<10):
                lines.append(c);            
                
        color = 0;
    
    return lines;

def getNonBlackVerticalLines(img):           
    
    color = 0;
    lines = [];
        
    for c in range(img.shape[1]):            
            for r in range(img.shape[0]):        
                if (img[r,c]!=0):
                    color=color+1;
                
                if (color>10):
                      lines.append(c);            
                
            color = 0;
    
    return lines;

def getBlackHorizontalLines(img):
    
    lines = [];
    cols = img.shape[1];
    
    for r in range(img.shape[0]):
        if (isBlackLine(img[r],cols) == True):
            lines.append(r);
            
    return lines;

def getNonBlackHorizontalLines(img):
    
    lines = [];
    cols = img.shape[1];
    
    for r in range(img.shape[0]):
        if (isBlackLine(img[r],cols) == False):
            lines.append(r);
            
    return lines;
    

    
def getStartingCrop(img, rows):    
    startingCrop = -1;    
    cols = img.shape[1];
    i=0;
    br=0;
    for x in range(rows):
        row = img[x];
        
        if (isBlackLine(row,cols)):
            startingCrop = startingCrop + 1;
        else:
            return startingCrop+1;        
    
    return startingCrop;

def getHeight(img):        
    height=0;    
    
    nonBlackLines = getNonBlackHorizontalLines(img);
    lastNonBlack = -1;
    firstNonBlack = -1;
    
    if (len(nonBlackLines) > 0 ):
              firstNonBlack = nonBlackLines[0];
              lastNonBlack = nonBlackLines[len(nonBlackLines)-1];
    
    height = lastNonBlack - firstNonBlack;
    
    return height;


def getWidth(img):    
    width = 0;        
    lastNonBlack = -1;
    firstNonBlack = -1;
    
    nonBlackLines = getNonBlackVerticalLines(img);
    
    if (len(nonBlackLines) > 0 ):
              firstNonBlack = nonBlackLines[0];
              lastNonBlack = nonBlackLines[len(nonBlackLines)-1];
    
    width = lastNonBlack - firstNonBlack;
    
    return width;


def _convertImg(imgFile):
    originalFile = imgFile.split('.');
    greyImgName = originalFile[0] + '_g' + '.' + originalFile[1];
    originalFilePath=IMG_PATH+'/'+imgFile;    
    img = cv2.imread(originalFilePath,0);   
    
    
    startingCrop = getStartingCrop(img, img.shape[0]);
    imgWidth = getWidth(img);
    imgHeight= getHeight(img)+startingCrop;
    
    #print(imgFile + ': ' + str(startingCrop) + '/' + str(imgWidth) + '/' + str(imgHeight));
    img3 = img[startingCrop:imgHeight,0:imgWidth];
    cv2.imwrite(IMG_PATH_CONVERTED+'/'+greyImgName,img3);
    
    imgOrig = cv2.imread(originalFilePath);
    img2 = imgOrig[startingCrop:imgHeight,0:imgWidth];
    cv2.imwrite(IMG_PATH_CONVERTED+'/'+imgFile,img2);    
    
def _convertImg2(imgFile):
    originalFile = imgFile.split('.');
    greyImgName = originalFile[0] + '_g' + '.' + originalFile[1];
    originalFilePath=IMG_PATH+'/'+imgFile;    
    img = cv2.imread(originalFilePath,0);   
    
    
    startingCrop = 20; #getStartingCrop(img, img.shape[0]);
    imgWidth = 288;#getWidth(img);
    imgHeight= 210; #getHeight(img)+startingCrop;
    
    #print(imgFile + ': ' + str(startingCrop) + '/' + str(imgWidth) + '/' + str(imgHeight));
    #img3 = img[startingCrop:imgHeight,0:imgWidth];
    #cv2.imwrite(IMG_PATH_CONVERTED+'/'+greyImgName,img3);
    
    imgOrig = cv2.imread(originalFilePath);
    img2 = imgOrig[startingCrop:imgHeight,0:imgWidth];
    cv2.imwrite(IMG_PATH_CONVERTED+'/'+imgFile,img2);    
    
def _convertImg3(imgFile):
    originalFile = imgFile.split('.');
    greyImgName = originalFile[0] + '_g' + '.' + originalFile[1];
    originalFilePath=IMG_PATH+'/'+imgFile;    
    img = cv2.imread(originalFilePath,0);   
    
    
    startingCrop = 60; #getStartingCrop(img, img.shape[0]);
    imgWidth = 288;#getWidth(img);
    imgHeight= 250;#getHeight(img)+startingCrop;
    
    #print(imgFile + ': ' + str(startingCrop) + '/' + str(imgWidth) + '/' + str(imgHeight));
    #img3 = img[startingCrop:imgHeight,0:imgWidth];
    #cv2.imwrite(IMG_PATH_CONVERTED+'/'+greyImgName,img3);
    
    imgOrig = cv2.imread(originalFilePath);
    img2 = imgOrig[startingCrop:imgHeight,0:imgWidth];
    cv2.imwrite(IMG_PATH_CONVERTED+'/'+imgFile,img2);
    
def hex_to_rgb(value):
    value = value.lstrip('#')
    #print('val'+value);
    lv = len(value)
    #print('lv:'+str(lv))
    if (lv // 3 == 0):
        return (0,0,0);
    return tuple(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3))    

def find_max_mode(list1):
    list_table = statistics._counts(list1)
    len_table = len(list_table)

    if len_table == 1:
        max_mode = statistics.mode(list1)
    else:
        new_list = []
        for i in range(len_table):
            new_list.append(list_table[i][0])
        max_mode = max(new_list) # use the max value here
    return max_mode

def _getColorTileAvg(tile,tileNum):
    
    tileCols = 16;
    tileRows = 10;    
    totalTilePixels = tileCols * tileRows;
    color = 0;
    index=0;
    rgbs = [];    
    r = 0;
    g = 0;
    b = 0;
    
    #RGB = (R*65536)+(G*256)+B , (when R is RED, G is GREEN and B is BLUE)
    for row in range(tileRows):          
            for col in range(tileCols):                        
                        r = r + tile[row-1,col-1][0];
                        g = g + tile[row-1,col-1][1];
                        b = b + tile[row-1,col-1][2];
                        #rgb =  (r*65536) + (g*256) + b;                          
                        #rgbs.append(rgb);                        
                        index = index + 1;               
    
    ravg=r // totalTilePixels;
    gavg=g // totalTilePixels;
    bavg=b // totalTilePixels;    
    
    avgColor = (ravg*65536) + (gavg*256) + bavg;  #find_max_mode(rgbs) #statistics.mode(rgbs);
        
    
    if (GENERATE_COLOR_TILES==1):
            for row in range(tileRows):
                  for col in range(tileCols):
                            tile[row-1,col-1][0] = ravg;
                            tile[row-1,col-1][1] = gavg;
                            tile[row-1,col-1][2] = bavg;
    
    
    return tile,avgColor;




def _getColorTile(tile,tileNum):
    
    tileCols = 16;
    tileRows = 10;    
    color = 0;
    index=0;
    rgbs = [];    
    
    #RGB = (R*65536)+(G*256)+B , (when R is RED, G is GREEN and B is BLUE)
    for row in range(tileRows):          
            for col in range(tileCols):                        
                        r = tile[row-1,col-1][0];
                        g = tile[row-1,col-1][1];
                        b = tile[row-1,col-1][2];
                        rgb =  (r*65536) + (g*256) + b;                          
                        rgbs.append(rgb);                        
                        index = index + 1;               
                     
    
    avgColor = find_max_mode(rgbs) #statistics.mode(rgbs);
    
    ravg=0;
    gavg=0;
    bavg=0;
    
    if (GENERATE_COLOR_TILES==1):
            avghx = hex(avgColor);                        
            avghx = str(avghx);                       
            avghx = '#'+avghx[2:len(avghx)];      
            avgrgb = hex_to_rgb(avghx);
            ravg = avgrgb[0];
            gavg = avgrgb[1];
            bavg = avgrgb[2];     
    
            for row in range(tileRows):
                  for col in range(tileCols):
                            tile[row-1,col-1][0] = ravg;
                            tile[row-1,col-1][1] = gavg;
                            tile[row-1,col-1][2] = bavg;
    
    
    return tile,avgColor;
    
            
    
    
def _getTile(img, tileX,tileY):
    
    
    
    startingX = (tileX)*TILE_WIDTH;
    startingY = (tileY)*TILE_HEIGHT;
    
    endingX = startingX+TILE_WIDTH;
    endingY = startingY+TILE_HEIGHT;
    
    if (endingX >= 288):
            endingX = 287;
    
    if (endingY >= 190):
            endingY = 189;    
    
    #print(str(startingY)+':'+str(endingY)+','+str(startingX)+':'+str(endingX));
    tile = img[startingY:endingY,startingX:endingX];
    
    return tile;
    
    
def _getTiles(img):               
    cols = 18;
    rows = 19;
    tiles = [];
        
    for r in range(rows):            
        for c in range(cols):        
            tile = _getTile(img,c,r);
            tiles.append(tile);                            
        
    
    return tiles;
    

def getTiles(imgFile):
    tiles = [];
    originalFile = imgFile.split('.');    
    originalFilePath=IMG_PATH+'/'+imgFile;    
    img = cv2.imread(originalFilePath);
    
    tiles = _getTiles(img);
    #print('tiles:' + str(len(tiles)));
    tileNum = 0;
    
    tileLine = imgFile+':';    
    tileData = '';
    
    for tile in tiles:
        #tileFileName = originalFile[0]+'.'+ str(originalFile[1]) + '.'+ str(tileNum) + '_t' + '.png';
        #if (cv2.empty(tile)==0):
        #cv2.imwrite(IMG_PATH_CONVERTED+'/'+tileFileName,tile); 
        
        #tileColorFileName = originalFile[0]+'.'+ str(originalFile[1]) + '.'+ str(tileNum) + '_c' + '.png';
        tileColor,tileAvgColor = _getColorTileAvg(tile,tileNum);
        #cv2.imwrite(IMG_PATH_CONVERTED+'/'+tileColorFileName,tileColor); 
        tileData = tileData+ str(tileAvgColor) + ",";
        #print(tileFileName);
        tileNum = tileNum + 1;
    
    print ('Total tiles for: ' + imgFile + ':' + str(tileNum));
    tileLine = tileLine+tileData[0:len(tileData)-1];
    
    return tileLine;


def convertImg(imgFile):
    originalFile = imgFile.split('.');
    originalFilePath=IMG_PATH+'/'+imgFile;
    croppedFilePath = IMG_PATH_CONVERTED+'/'+imgFile;
    total=0;
    #print(originalFilePath);
    #print(originalFile[0][0:3]);
    if (os.path.exists(croppedFilePath)):
                                total=total;
    else:
        _convertImg3(imgFile);
        total = total + 1;
    return total;
    #if (originalFile[0][-1]!='g'):
        #if (originalFile[0][0:3]=='img'):
         #               _convertImg2(imgFile);
        #if (originalFile[0][0:3]!='img'):
         #               _convertImg3(imgFile);    
                
        
    

In [30]:
def setPoint(center,file):
    img = cv2.imread(IMG_PATH+'/'+file);
    #print(tile[0]);
    row=0;
    col=0;
    tileCols = 16;
    tileRows = 10;
    index=0;
    centerX = int(center) % 288;
    centerY = int(center) // 288;


def genImgCsv(center,imgColors,file):    
    img = cv2.imread(IMG_PATH+'/'+file);
    print('Generating tiles for' + IMG_PATH+'/'+file);
    row=0;
    col=0;
    tileCols = 16;
    tileRows = 10;
    index=0;
    centerX = int(center) % 288;
    centerY = int(center) // 288;
    
    print('Colors: ' + str(len(imgColors)));
    for avgColor in imgColors: 
            avghx = hex(int(avgColor));                        
            avghx = str(avghx);                       
            avghx = '#'+avghx[2:len(avghx)];      
            avgrgb = hex_to_rgb(avghx);
            ravg = avgrgb[0];
            gavg = avgrgb[1];
            bavg = avgrgb[2];
            
            row = index // 18;
            col = index % 18;
            
            startingX = (col)*TILE_WIDTH;
            startingY = (row)*TILE_HEIGHT;
    
            endingX = startingX+TILE_WIDTH;
            endingY = startingY+TILE_HEIGHT;
            
            
    
            if (endingX >= 288):
                endingX = 287;
    
            if (endingY >= 190):
                endingY = 189;    
    
            #print(str(startingY)+':'+str(endingY)+','+str(startingX)+':'+str(endingX));
            tile = img[startingY:endingY,startingX:endingX];
            
            for row in range(tileRows):
                   for col in range(tileCols):
                            tile[row-1,col-1][0] = ravg;
                            tile[row-1,col-1][1] = gavg;
                            tile[row-1,col-1][2] = bavg;           
            
            
            img[startingY:endingY,startingX:endingX] = tile;
            #img[centerY:10,centerX:10] = [255,0,0];
            img.itemset((centerY,centerX,2),255);
            img.itemset((centerY,centerX,1),0);
            img.itemset((centerY,centerX,0),0);
            
            #print('index:'+str(index)+"="+str(row)+','+str(col)+":"+str(avgColor)+'\r');           
            index = index + 1;
            
    print('img: '+file+ ', center: '+ center + 'centerX:' + str(centerX) + ', centerY' + str(centerY));
    return img;
            
    
def getImgCsv():
    total=0;
    with open(CSV_PATH, 'rU') as csvfile:
                csvline = csv.reader(csvfile);
                for row in csvline:
                    if (total<1): 
                             print(str(len(row)));
                             img=genImgCsv(row[len(row)-1],row[1:len(row)-1],row[0]);
                             cv2.imwrite('tile'+str(total)+'.png',img); 
                    total=total+1;
                    

In [32]:
def genTileSet():
    total = 0;
    ftileSet = open('tilesetAvg.csv','w');
    for i in range(0,10):
            f = onlyfiles[i];
            tileLine=getTiles(f);
            ftileSet.write(tileLine+'\r');
            total =  total + 1;
            print('Gen tiles for: ' + f)
    ftileSet.close();        
    print ('Total files:' + str(total));        

#genTileSet();
getImgCsv();

344
Generating tiles forC:/models/ryuboxes/training/RuyCunclilllasDerecha1.0.png
Colors: 342
img: RuyCunclilllasDerecha1.0.png, center: 46548centerX:180, centerY161


C:\anaconda\lib\site-packages\ipykernel_launcher.py:76: DeprecationWarning: 'U' mode is deprecated


In [60]:
def getSprite(center, imgFile):
  img = cv2.imread(imgFile)
  centerX = int(center) % 288;
  centerY = int(center) // 288;
  sw = 100
  sh = 100
  boxes = numpy.array(3)
  boxes = [[centerX-40,centerY-40,centerX+20,centerY+20]]  
  imgBox1 = cv2.rectangle(img,(boxes[0][0],boxes[0][1]),(boxes[0][2],boxes[0][3]),(0,0,255),1)  
  boxes2 = [[boxes[0][0]+60,boxes[0][1],boxes[0][2]+60,boxes[0][3]]]
  imgBox2 = cv2.rectangle(imgBox1,(boxes2[0][0],boxes2[0][1]),(boxes2[0][2],boxes2[0][3]),(255,0,0),1)
  boxes3 = [[boxes[0][0]-10,boxes[0][1]-20,boxes[0][2]+60,boxes[0][3]+10]]   
  imgBox3 = cv2.rectangle(imgBox2,(boxes3[0][0],boxes3[0][1]),(boxes3[0][2],boxes3[0][3]),(0,255,0),1)             
  boxes4 = [[boxes[0][0]-20,boxes[0][1]-10,boxes[0][2]+10,boxes[0][3]+10]]   
  imgBox4 = cv2.rectangle(imgBox3,(boxes4[0][0],boxes4[0][1]),(boxes4[0][2],boxes4[0][3]),(255,255,255),1)                 
  cv2.imwrite('box1.png',imgBox4)

  spr1 = cv2.imread(imgFile)
  spr1 = spr1[boxes[0][1]:boxes[0][3],boxes[0][0]:boxes[0][2]];
  cv2.imwrite('spr1.png',spr1)

  spr2 = cv2.imread(imgFile)
  spr2 = spr2[boxes2[0][1]:boxes2[0][3],boxes2[0][0]:boxes2[0][2]];
  cv2.imwrite('spr2.png',spr2)

  spr3 = cv2.imread(imgFile)
  spr3 = spr3[boxes3[0][1]:boxes3[0][3],boxes3[0][0]:boxes3[0][2]];
  cv2.imwrite('spr3.png',spr3)
    
  spr4 = cv2.imread(imgFile)
  spr4 = spr4[boxes4[0][1]:boxes4[0][3],boxes4[0][0]:boxes4[0][2]];
  cv2.imwrite('spr4.png',spr4)    

  imgCircle = cv2.circle(imgBox4,(centerX,centerY), 2, (0,0,255), -1)
  cv2.imwrite('circle.png',imgCircle)

    

getSprite(42135,'point.png')

    